In [ ]:
from bs4 import BeautifulSoup
import requests
import json
import re

url = 'https://www.aonprd.com/'

class ArchetypeScraper:
    def __init__(self, url):
        self.url = url
        self.href = f'CavalierOrders.aspx'
        self.page = requests.get(url + self.href)
        self.soup = BeautifulSoup(self.page.content, 'html.parser')
        self.table = self.soup.find("table")


    def clean_text(self, text):
        # Replace Unicode right single quotation mark with an apostrophe
        cleaned_text = [line.replace('\u2019', "'").replace('\u2018', "'").replace('\u201c', '"').replace('\u201d', '"') for line in text.split('\n') if line.strip() and line.strip() != "."]
        return ' '.join(cleaned_text).strip()
    
    def remove_parenthesis(self, input_string):
        pattern = r"\(.*?pg\..*?\)"
        result = re.sub(pattern, '', input_string)
        return result
    
    def link_searcher(self,a_tags):

        self.href2_list = []

        for a in a_tags:
            # Check if the link contains "CavalierOrders.aspx"
            if "CavalierOrders.aspx" in str(a):
                # Extract everything within double quotes
                regex = r'href="(.*?)"'
                href_unclean = str(a)
                match = re.search(regex, href_unclean)

                if match:
                    self.href2 = match.group(1)
                    self.href2_list.append(self.href2)

        # Print or return self.href2_list as needed
        print(self.href2_list)


        return self.href2 
    
    def get_order_name(self):
        self.order_names = []
        for href2 in self.href2_list:
            match = re.search(r'ItemName=([^&]*)', href2)
            if match:
                order_name = match.group(1)
                self.order_names.append(order_name)

        return self.order_names

# What we want to do is to get the data once we see a <b> tag, use the <b> tag as the key section
# Since it's going to be the same each time we can manually set them (like in the items json file)
    def span_search(self):
        for i, href2 in enumerate(self.href2_list):
            self.detail_page = requests.get(url + href2)
            self.detail_soup = BeautifulSoup(self.detail_page.content, 'html.parser')

            # Search for the span tag in the detail_soup
            span_tag = self.detail_soup.find('span', id="ctl00_MainContent_DataListTypes_ctl00_LabelName")

            # Check if span_tag is found
            if span_tag:
                # Create a dictionary to store key-value pairs
                span_data = {}

                # Iterate through the contents of the span_tag
                for child in span_tag.children:
                    # Check if the child is a <b> or <i> tag
                    if child.name == 'b' or child.name == 'i':
                        span_data[child.text.strip()] = "/n"
                    else:
                        span_data[child.text.strip()] = child.text.strip()

                # Use the order name as the title
                title = f'"{self.order_names[i]}"'

                # Print the title and dictionary as a JSON-formatted string with indentation
                print(f"{title}: {json.dumps(span_data, indent=2)},")
            

    def get_class_info(self):
        span_tag = self.soup.find('span', id="ctl00_MainContent_SubHeader")
        print(span_tag)
        a_tags = list(span_tag.find_all('a'))
        print(type(a_tags))
        print(a_tags)

        self.link_searcher(a_tags)
        self.get_order_name()
        self.span_search()            

        #now we have all the a_tags

        # want to grab everything in this span list:


        # if span_tag:
        #     # Extract all text content within the span
        #     span_content = span_tag.get_text(strip=True)
        #     print(f'Span content: {span_content}')

        #     # Define patterns to split the text
        #     patterns = ['StrengthNormal', 'StrengthGreater', 'TypeExploit of the Outer Rifts']

        #     # Create a regular expression pattern for splitting
        #     split_pattern = '|'.join(map(re.escape, patterns))

        #     # Split the text based on the patterns
        #     separated_sections = re.split(split_pattern, span_content)

        #     # Print each separated section along with exploit names
        #     # Need to skip the first section of separated sections, because it was the wrong info
        #     for section, exploit_name in zip(separated_sections[1:], self.exploit_data):
        #         cleaned_section = self.clean_text(section)
        #         cleaned_section = self.remove_parenthesis(cleaned_section)

        #         exploit_data[exploit_name.strip()] = [cleaned_section.strip()]

        #     json_output = json.dumps(exploit_data, indent=2)

        #     # Print the JSON-formatted string
        #     print(json_output)        

        

          
       
    

# Instantiate the ArchetypeScraper class
scraper_instance = ArchetypeScraper(url)

# Call the get_class_info method
scraper_instance.get_class_info()